In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import log_loss
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
import pickle
import re
  

train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_y = train[list_classes].values
train['comment_text'] = train['comment_text'].fillna('nan')
test['comment_text'] = test['comment_text'].fillna('nan')
print('load done')

load done


In [3]:
tf_vec1 = TfidfVectorizer(sublinear_tf=True,
                            strip_accents='unicode',
                            analyzer='word',
                            token_pattern=r'\w{1,}',
                            ngram_range=(1, 1),
                            max_features=15000)
train_tfidf1 = tf_vec1.fit_transform(train['comment_text'].values)
test_tfidf1 = tf_vec1.transform(test['comment_text'].values)

print(train_tfidf1.shape)

(159571, 15000)


In [4]:
n_comp = 30
svd_obj = TruncatedSVD(n_components=n_comp, algorithm='arpack')
train_svd1 = svd_obj.fit_transform(train_tfidf1)
test_svd1 = svd_obj.transform(test_tfidf1)

print(type(train_svd1),train_svd1.shape)
with open('../features/tfidf_feat1.pkl','wb') as fout:
    pickle.dump([train_svd1,test_svd1],fout)
print('dump done')
#simple_eval(train_svd1,train_y,XGBClassifier)
print('-----------')

<class 'numpy.ndarray'> (159571, 30)
dump done
-----------


In [5]:
#simple_eval(train_tfidf1,train_y,LogisticRegression)
#print('-----------')

In [6]:
tf_vec2 = TfidfVectorizer(sublinear_tf=True,
                        strip_accents='unicode',
                        analyzer='char',
                        ngram_range=(1, 5),
                        max_features=20000
                         )
train_tfidf2 = tf_vec2.fit_transform(train['comment_text'].values)
test_tfidf2 = tf_vec2.transform(test['comment_text'].values)
print(train_tfidf2.shape)

(159571, 20000)


In [7]:
svd_obj = TruncatedSVD(n_components=n_comp, algorithm='arpack')
train_svd2 = svd_obj.fit_transform(train_tfidf2)
test_svd2 = svd_obj.transform(test_tfidf2)

print(type(train_svd2),train_svd2.shape)
with open('../features/tfidf_feat2.pkl','wb') as fout:
    pickle.dump([train_svd2,test_svd2],fout)
print('dump done')
#simple_eval(train_svd2,train_y,XGBClassifier)
print('-----------')

<class 'numpy.ndarray'> (159571, 30)
dump done
-----------


In [14]:
test_m = LogisticRegression()
test_m.fit(train_tfidf1,train_y[:,0])
res = test_m.predict_proba(train_tfidf1)
print(res[:20])

[[  9.93791465e-01   6.20853525e-03]
 [  9.80035544e-01   1.99644562e-02]
 [  9.78047041e-01   2.19529591e-02]
 [  9.98319365e-01   1.68063457e-03]
 [  9.29023406e-01   7.09765939e-02]
 [  9.93445038e-01   6.55496179e-03]
 [  7.24703146e-02   9.27529685e-01]
 [  9.41548506e-01   5.84514941e-02]
 [  9.68103493e-01   3.18965074e-02]
 [  9.72104652e-01   2.78953476e-02]
 [  9.99899362e-01   1.00637849e-04]
 [  9.44155988e-01   5.58440124e-02]
 [  9.45490459e-01   5.45095412e-02]
 [  9.77286823e-01   2.27131766e-02]
 [  9.65986600e-01   3.40134002e-02]
 [  9.39576365e-01   6.04236353e-02]
 [  8.59454509e-01   1.40545491e-01]
 [  9.97199600e-01   2.80039959e-03]
 [  9.95466470e-01   4.53352967e-03]
 [  9.87808196e-01   1.21918045e-02]]


In [24]:
from sklearn.model_selection import KFold
def gen_base_lr_feat(train_x,train_y,test_x,fold_cnt=3,rnd=1):
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((159571,6)),np.zeros((153164,6))
    for train_index, test_index in kf.split(train_x):
        # x,y
        curr_x,curr_y = train_x[train_index],train_y[train_index]
        hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
        
        for i in range(6):
            model = LogisticRegression()
            # train and pred
            # fit for i
            model.fit(curr_x, curr_y[:,i])
            
            # prepare for i on this fold
            hold_out_pred = model.predict_proba(hold_out_x)
            #print(hold_out_pred[:10])
            curr_train_pred = model.predict_proba(curr_x)
            print(i,log_loss(hold_out_y[:,i],hold_out_pred),log_loss(curr_y[:,i],curr_train_pred))
            train_pred[test_index,i] = list(hold_out_pred[:,1].flatten())
            print('value', np.sum(train_pred[:,i]))
            
            # prepare test
            y_test = model.predict_proba(test_x)[:,1]
            test_pred[:,i] += y_test
        print('===========this fold done')
    test_pred = test_pred / fold_cnt
    #print(train_pred[:10])
    return [train_pred, test_pred]

with open('../features/lr_feat1.pkl','wb') as fout:
    lr_feat1 = gen_base_lr_feat(train_tfidf1,train_y,test_tfidf1,3,rnd=3)
    pickle.dump(lr_feat1,fout)


0 0.116384088302 0.101482820947
value 5075.21203005
1 0.0268826085073 0.0235620319907
value 530.08836994
2 0.0654563051945 0.0551811376693
value 2799.9523342
3 0.011626509413 0.00880569132737
value 150.85418302
4 0.0781025411574 0.0672601955517
value 2628.96653389
5 0.0268439722249 0.0237216332192
value 464.804110171
===========this fold done
0 0.11572313302 0.10150259587
value 10092.4437629
1 0.0271613106961 0.0233816389652
value 1051.08754105
2 0.0643928970435 0.0564350109428
value 5557.40135751
3 0.0105003678193 0.00883133575316
value 314.530206973
4 0.0786079311691 0.0675855178469
value 5181.41206089
5 0.0267731474314 0.0233628307555
value 935.816747437
===========this fold done
0 0.11597592706 0.10152805768
value 15110.1583416
1 0.027288950596 0.0232874378838
value 1575.20882431
2 0.0646987787613 0.056046521607
value 8321.9239402
3 0.00989523203243 0.00926211488746
value 472.52580499
4 0.0769600187191 0.0680101814068
value 7755.71244678
5 0.02857696181 0.0229811719919
value 1377.4

In [25]:
try:
    del tf_vec2,tf_vec1,svd_obj
except:
    pass
from scipy.sparse import hstack
from scipy.sparse import csr_matrix
print(type(train_tfidf2),train_tfidf2.shape)
comb_train = csr_matrix(hstack((train_tfidf2,train_tfidf1)))
print(type(comb_train),comb_train.shape)
comb_test = csr_matrix(hstack((test_tfidf2,test_tfidf1)))

<class 'scipy.sparse.csr.csr_matrix'> (159571, 20000)
<class 'scipy.sparse.csr.csr_matrix'> (159571, 35000)


In [26]:
with open('../features/lr_feat2.pkl','wb') as fout:
    lr_feat2 = gen_base_lr_feat(comb_train,
                                train_y,
                                comb_test,3,rnd=11)
    pickle.dump(lr_feat2,fout)

0 0.102241368889 0.0801105597102
value 5060.86889559
1 0.0246173591039 0.0192587631549
value 526.726637916
2 0.056389939991 0.0419326132626
value 2788.22846853
3 0.0102994295764 0.00665408777849
value 148.649550485
4 0.0701977896968 0.0548491407929
value 2614.47061102
5 0.0234527531648 0.0180436061952
value 462.585903392
===========this fold done
0 0.10136985833 0.080415887666
value 10038.1756267
1 0.0252233257442 0.018949473646
value 1043.39458345
2 0.0551359317843 0.0432470185548
value 5509.11006496
3 0.00938403602261 0.00670760995522
value 308.202739151
4 0.0715143483368 0.0546542406887
value 5146.86209884
5 0.0239251877493 0.0175806382771
value 920.281007185
===========this fold done
0 0.10195112328 0.0801885522765
value 15047.5779806
1 0.0252177777601 0.0188944735427
value 1558.15478185
2 0.0557356525351 0.0428063797512
value 8260.09545158
3 0.00875962543946 0.00711652538281
value 457.050587866
4 0.0697786563356 0.0550544375666
value 7699.14804546
5 0.0247902528441 0.017645077146


In [27]:
from sklearn.naive_bayes import MultinomialNB
def gen_base_mnb_feat(train_x,train_y,test_x,fold_cnt=3,rnd=1):
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((159571,6)),np.zeros((153164,6))
    for train_index, test_index in kf.split(train_x):
        # x,y
        curr_x,curr_y = train_x[train_index],train_y[train_index]
        hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
        
        for i in range(6):
            model = MultinomialNB(alpha=0.2)
            # train and pred
            # fit for i
            model.fit(curr_x, curr_y[:,i])
            
            # prepare for i on this fold
            hold_out_pred = model.predict_proba(hold_out_x)
            curr_train_pred = model.predict_proba(curr_x)
            print(i,log_loss(hold_out_y[:,i],hold_out_pred),log_loss(curr_y[:,i],curr_train_pred))
            train_pred[test_index,i] = hold_out_pred[:,1]
            
            # prepare test
            y_test = model.predict_proba(test_x)[:,1]
            test_pred[:,i] += y_test
        print('===========this fold done')
    test_pred = test_pred / fold_cnt
    #print(train_pred[:10])
    return [train_pred, test_pred]

with open('../features/mnb_feat1.pkl','wb') as fout:
    _feat1 = gen_base_mnb_feat(train_tfidf1,train_y,test_tfidf1,3)
    pickle.dump(_feat1,fout)

0 0.135064637786 0.119426880993
1 0.0280498515944 0.022540605607
2 0.0822088496144 0.070409846418
3 0.0174418668927 0.0114750738812
4 0.0859010193469 0.0748272828438
5 0.0292018964185 0.0244956319078
===========this fold done
0 0.136402005569 0.119277291188
1 0.0278692392234 0.0225375090818
2 0.0831849543181 0.0708722878025
3 0.0140445220646 0.0120016718168
4 0.0884407651275 0.0742108440618
5 0.0303573563701 0.0239013261819
===========this fold done
0 0.135258956974 0.119395227808
1 0.0278011896406 0.0227454464303
2 0.0829899448268 0.0708068473162
3 0.0142391699087 0.0119526680429
4 0.087647809208 0.0748190278716
5 0.0322763447175 0.0235071268866
===========this fold done


In [28]:
with open('../features/mnb_feat2.pkl','wb') as fout:
    _feat2 = gen_base_mnb_feat(comb_train,
                                train_y,
                                comb_test,3,rnd=29)
    pickle.dump(_feat2,fout)

0 0.186172501784 0.17796216859
1 0.138356732746 0.139951829086
2 0.156714015863 0.143592378619
3 0.0372164529446 0.0324088537377
4 0.180579797164 0.167767305069
5 0.121608716466 0.117026055627
===========this fold done
0 0.19673424959 0.176980401368
1 0.152391556595 0.138566231187
2 0.149583943395 0.143122222898
3 0.0420097037746 0.0370923417515
4 0.176807723136 0.165516545156
5 0.1301960023 0.11808540472
===========this fold done
0 0.196710879649 0.178708104766
1 0.141026969453 0.14040481386
2 0.158906866955 0.144656850901
3 0.0454890042183 0.0409103477265
4 0.181096771759 0.16944895783
5 0.117908591271 0.112924617458
===========this fold done
